SUST Bangla

# 7 Emotion Spiker Independent

In [ ]:
#Data Process And save
from numpy.core.fromnumeric import shape
import librosa
import pathlib
import numpy as np
import pandas as pd
import csv
import pywt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from google.colab import drive0

def get_cwt_mel(path, n_fft, hop_length, n_mels):
    y, sr = librosa.load(path,sr=16000,duration=8)
    file_length = np.size(y)
    if file_length < 128000:
        y = np.concatenate((y, np.zeros(128000-file_length)), axis=0)
    else:
        y=y[0:128000]

    #mel_spectrogram = librosa.feature.melspectrogram(y, sr, n_fft=2048, hop_length=1001, n_mels=128)
    mfcc = librosa.feature.mfcc(y, sr, hop_length=501,n_mfcc=128)
    stft = np.abs(librosa.stft(y,n_fft=254, hop_length=501))
    chroma=librosa.feature.chroma_stft(S=stft,n_chroma=128)
    log_mel_spectrogram = np.concatenate((stft,mfcc,chroma), axis=1)
    log_mel_spectrogram = log_mel_spectrogram.reshape((-1,))
    return log_mel_spectrogram


def classify_files(path):
    dataset_dict = {
        'total': 0,
        'file_dict': {
            'ANGRY': {'represent': 0, 'count': 0, 'all_data': []},
            'DISGUST': {'represent': 1, 'count': 0, 'all_data': []},
            'FEAR': {'represent': 2, 'count': 0, 'all_data': []},
            'HAPPY': {'represent': 3, 'count': 0, 'all_data': []},
            'NEUTRAL': {'represent': 4, 'count': 0, 'all_data': []},
            'SAD': {'represent': 5, 'count': 0, 'all_data': []},
            'SURPRISE': {'represent': 6, 'count': 0, 'all_data': []}
        }
    }    

   
    wav_path = pathlib.Path(path)
    emotion_file_list = [str(file_name) for file_name in wav_path.glob('*.wav')]

    p = len(str(wav_path))

    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        print(emotion_label)
        
        emotion_classify_file_list = [letter for letter in emotion_file_list if letter.find(emotion_label)!=-1]

        print(emotion_classify_file_list)
        files_count = len(emotion_classify_file_list)

        dataset_dict['file_dict'][emotion_label]['count'] = files_count
        dataset_dict['total'] = dataset_dict['total'] + files_count
        
        emotion_data = [get_cwt_mel(path, n_fft=2048, hop_length=512, n_mels=128)

        
        for path in emotion_classify_file_list] 
        print(shape(emotion_data))
        dataset_dict['file_dict'][emotion_label]['all_data'] = emotion_data 

    return dataset_dict

def load_data(path):

    train_data_x = []
    train_data_y = []
    validation_data_x = []
    validation_data_y = []
    test_data_x = []
    test_data_y = []

    dataset_dict = classify_files(path)

    '''Split data set'''
    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        x = dataset_dict['file_dict'][emotion_label]['all_data']
        count = dataset_dict['file_dict'][emotion_label]['count']
        y = np.full(count, dataset_dict['file_dict'][emotion_label]['represent'])

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.80, random_state=1)

        train_data_x = np.append(train_data_x, x_train)
        train_data_y = np.append(train_data_y, y_train)

        test_data_x = np.append(test_data_x, x_test)
        test_data_y = np.append(test_data_y, y_test)
    '''
    train_data_x=np.array(train_data_x).reshape(len(train_data_y),-1)
    with open(path +'/data/train_data_x.csv',"w+") as my_csv:
      csvWriter = csv.writer(my_csv,delimiter=',')
      csvWriter.writerows(train_data_x)

    test_data_x=np.array(test_data_x).reshape(len(test_data_y),-1)
    with open(path +'/data/test_data_x.csv',"w+") as my_csv:
      csvWriter = csv.writer(my_csv,delimiter=',')
      csvWriter.writerows(test_data_x)
    ''' 
   

    #np.savetxt(path +'/data/train_data_x.csv', np.array(train_data_x))
    np.savetxt(path +'/data/train_data_y.csv', train_data_y, delimiter=',')
   #np.savetxt(path +'/data/test_data_x.csv', test_data_x, delimiter=',')
    np.savetxt(path +'/data/test_data_y.csv', test_data_y, delimiter=',')
    
    x=128 #128
    y=128#768 #384 #256
    z=5
    train_data_x = np.array(train_data_x).reshape(-1, x, y,3)
    #train_data_x = np.array(train_data_x).reshape(-1, x, y, z,1)
    train_data_y = np.array(train_data_y)
    #test_data_x = np.array(test_data_x).reshape(-1, x, y, z,1)
    test_data_x = np.array(test_data_x).reshape(-1, x, y,3)
    test_data_y = np.array(test_data_y)

    return train_data_x,train_data_y,test_data_x,test_data_y 

def classify_files_new(path):
    dataset_dict = {
        'total': 0,
        'file_dict': {
            'ANGRY': {'represent': 0, 'count': 0, 'all_data': []},
            'DISGUST': {'represent': 1, 'count': 0, 'all_data': []},
            'FEAR': {'represent': 2, 'count': 0, 'all_data': []},
            'HAPPY': {'represent': 3, 'count': 0, 'all_data': []},
            'NEUTRAL': {'represent': 4, 'count': 0, 'all_data': []},
            'SAD': {'represent': 5, 'count': 0, 'all_data': []},
            'SURPRISE': {'represent': 6, 'count': 0, 'all_data': []}
        }
    }

    wav_path = pathlib.Path(path)
    emotion_file_list = [str(file_name) for file_name in wav_path.glob('*.wav')]

    p = len(str(wav_path))
    train_data_x = []
    train_data_y = []
    test_data_x = []
    test_data_y = []
    total = []
    temp = 0
    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        print(emotion_label)

        emotion_classify_file_list = [letter for letter in emotion_file_list if letter.find(emotion_label) != -1]

        #print(emotion_classify_file_list)
        files_count = len(emotion_classify_file_list)

        dataset_dict['file_dict'][emotion_label]['count'] = files_count
        dataset_dict['total'] = dataset_dict['total'] + files_count

        emotion_data = [get_cwt_mel(path, n_fft=2048, hop_length=512, n_mels=128) for path in
                        emotion_classify_file_list]

        print(shape(emotion_data))
        # dataset_dict['file_dict'][emotion_label]['all_data'] = emotion_data

        x = emotion_data
        count = dataset_dict['file_dict'][emotion_label]['count']
        y = np.full(count, dataset_dict['file_dict'][emotion_label]['represent'])
        dataset_dict['file_dict'][emotion_label]['represent'] = []
        dataset_dict['file_dict'][emotion_label]['count'] = []
        emotion_data = []
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.80, random_state=1)

        x = []
        y = []
        x = 128
        y = 256
        z = 3
        print(emotion_label)
        train_data_x = np.append(train_data_x, np.array(x_train)).reshape(-1, x, y, z)
        print(train_data_x.shape)
        test_data_x = np.append(test_data_x, np.array(x_test)).reshape(-1, x, y, z)
        print(test_data_x.shape)
        
        
        # print(total.shape)
        train_data_y = np.append(train_data_y, y_train)
        test_data_y = np.append(test_data_y, y_test)
        
        '''
        print(emotion_label)
        x_train=np.array(x_train).reshape(len(y_train),-1)
        with open(path +'/data/'+ emotion_label+'_train_data_x.csv',"w+") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            csvWriter.writerows(x_train)
        my_csv=[]
        '''
        x_train=[]
        y_train = []
        '''
        x_test=np.array(x_test).reshape(len(y_test),-1)
        with open(path +'/data/'+ emotion_label+'_test_data_x.csv',"w+") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            csvWriter.writerows(x_test)
        my_csv = []
       '''     
        x_test=[]
        y_test = []
        
    np.savetxt(path + '/data/train_data_y1.csv', train_data_y, delimiter=',')
    np.savetxt(path + '/data/test_data_y1.csv', test_data_y, delimiter=',')
    #train_data_y = []
    #test_data_y = []
    return train_data_x, train_data_y, test_data_x, test_data_y




path = 'D:/Education/MSc KUET/Thesis/Dataset/SUBESCO'
path='/content/drive/MyDrive/MSc KUET/Thesis/Dataset//SUBESCO'
# train_data_x,train_data_y,test_data_x,test_data_y =load_data(path)
# train_data_x,train_data_y,test_data_x,test_data_y =load_data_Paralle(path)
train_data_x, train_data_y, test_data_x, test_data_y = classify_files_new(path)






In [ ]:
import numpy as np
def splitCVSData(path,firstIndex,lastIndex):
  with open(path) as fd:
    reader=csv.reader(fd)
    print(reader)
    return np.array([row for idx, row in enumerate(reader) if idx in range (firstIndex,lastIndex)])     
  

def dataReshap(path):
    x=128 #128
    y=256#384 #256
    #train_data_x = np.array(np.genfromtxt(path +'/data/train_data_x.csv', delimiter=',')).reshape(-1, 128, 256, 8,1)
    #test_data_x = np.array(np.genfromtxt(path +'/data/test_data_x.csv', delimiter=',')).reshape(-1, 128, 256, 8,1)
    #temp=np.genfromtxt(path +'/data/train_data_x.csv', delimiter=',')
    #print(temp[0])

    train_data_x = np.array(np.genfromtxt(path +'/data/train_data_x.csv', delimiter=','))
    print('train_data_x.csv')
    train_data_y = np.array(np.genfromtxt(path +'/data/train_data_y.csv', delimiter=','))
    print('train_data_y.csv')
   
    test_data_x = np.array(np.genfromtxt(path +'/data/test_data_x.csv', delimiter=','))
    print('test_data_x.csv')
    test_data_y = np.array(np.genfromtxt(path +'/data/test_data_y.csv', delimiter=','))
    print('test_data_y.csv')

    #return train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y
    return train_data_x.reshape(-1, x, y,3), train_data_y,test_data_x.reshape(-1, x, y, 3), test_data_y

def dataReshapNew(path):
    x=128 #128
    y=513#384 #256
    emotion=['ANGRY','DISGUST','FEAR','HAPPY','NEUTRAL','SAD','SURPRISE']
    print(emotion[0])
    train_data_x =np.array(np.genfromtxt(path +'/data/'+ emotion[0]+'_train_data_x.csv', delimiter=','))
    print(train_data_x.shape)
    for i in range(1,len(emotion)):
      print(emotion[i])
      train_data_x =np.concatenate(train_data_x, np.array(np.genfromtxt(path +'/data/'+ emotion[i]+'_train_data_x.csv', delimiter=',')))
      print(train_data_x.shape)
    
    
    '''
    train_data_y = np.array(np.genfromtxt(path +'/data/train_data_y.csv', delimiter=','))
    print('train_data_y.csv')
   
    test_data_x = np.array(np.genfromtxt(path +'/data/test_data_x.csv', delimiter=','))
    print('test_data_x.csv')
    test_data_y = np.array(np.genfromtxt(path +'/data/test_data_y.csv', delimiter=','))
    print('test_data_y.csv')

    #return train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y
    return train_data_x.reshape(-1, x, y,3), train_data_y,test_data_x.reshape(-1, x, y, 3), test_data_y
    '''

path = '/content/drive/Othercomputers/My Laptop 07-10-2021/MSc KUET/Thesis/Dataset/RAVDESS/Speech/Actor_01'
path = '/content/drive/Othercomputers/My Laptop 07-10-2021/MSc KUET/Thesis/Dataset/RAVDESS/All'
path='/content/drive/MyDrive/MSc KUET/Thesis/Dataset/RAVDESS/All'

path='/content/drive/MyDrive/MSc KUET/Thesis/Dataset//SUBESCO'
dataReshapNew(path)

##Model SUST

In [ ]:
from numpy.core.fromnumeric import shape
import librosa
import pathlib
import numpy as np
import pywt
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow import keras,nn
from tensorflow.keras import layers

def model3d(input_shape, num_classes):

    model = keras.Sequential(name='model3d')

    #LFLB1
    model.add(layers.Conv3D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
    
    #LFLB2
    model.add(layers.Conv3D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB3
    model.add(layers.Conv3D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    
    #LFLB4
    model.add(layers.Conv3D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    #LSTM
    #model.add(layers.LSTM(256))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    
    #model.add(keras.layers.Dense(128,activation=nn.relu))
    #model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model



def model2dv2(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=(3,3),strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    
    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=(3,3),strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=(3,3),strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))


    
    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=(3,3),strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    #LSTM
    #model.add(layers.LSTM(256))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    
    #model.add(keras.layers.Dense(128,activation=nn.relu))
    #model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model


def model2d(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    
    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))


    
    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    #LSTM
    #model.add(layers.LSTM(256))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    
    #model.add(keras.layers.Dense(128,activation=nn.relu))
    #model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model

def model2CNN2F(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB3
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=8, strides=8))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model


#physical_device = tf.config.experimental.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_device[0], True)




#def train(train_data_x, train_data_y, validation_data_x, validation_data_y,emotion,emotionNumber):
def train(train_data_x, train_data_y,emotion,emotionNumber):
    #model = model2dv2(input_shape=(128, 512, 1), num_classes=emotionNumber)
    model = model2d(input_shape=(128,256, 3), num_classes=emotionNumber)
    #model = model3d(input_shape=(128,128,3,1), num_classes=emotionNumber)
    
    #model = model2CNN2F(input_shape=(128, 256, 1), num_classes=emotionNumber)
    
    model.summary()
    es = EarlyStopping(monitor='val_categorical_accuracy',mode='max',verbose=1,patience=20)

    mc = ModelCheckpoint(path+'/model/'+emotion+'_max_model1.h5',monitor='val_categorical_accuracy',mode='max',verbose=1,save_best_only=True)
    history=model.fit(train_data_x, train_data_y,validation_data=(test_data_x, test_data_y),epochs=120,batch_size=10,verbose=2,callbacks=[es,mc])
    #vacc=history.history['val_categorical_accuracy'][len(history.history['val_categorical_accuracy'])-1]
    #mc = ModelCheckpoint(path+'/model/'+emotion+'_model.h5',mode='max',verbose=0,save_best_only=True)
    #history=model.fit(train_data_x, train_data_y,epochs=50,batch_size=20,verbose=2,callbacks=[mc])
    acc=history.history['categorical_accuracy'][len(history.history['categorical_accuracy']) - 1]
    #model.save(path+'/model/'+emotion+'_model.h5')
    return acc




def test(test_data_x, test_data_y,emotion):
    
    new_model = load_model(path+'/model/'+emotion+'_max_model.h5')
    history=new_model.evaluate(test_data_x, test_data_y, batch_size=1)
    predict=new_model.predict(test_data_x)
    return history[1]

def maxIndex(data):
  max=data[0]
  index=0
  for i in range(1,len(data)):
    if(max<data[i]):
      max=data[i]
      index=i
  return index  

def test_emotion(test_data_x, test_data_y,total,emotion):
    
    new_model = load_model(path+'/model/'+emotion+'_max_model.h5')
    #test_data_y=to_categorical(test_data_y)
    history=new_model.evaluate(test_data_x, test_data_y, batch_size=10)
    predict=new_model.predict(test_data_x)
    test_data_y=np.argmax(test_data_y, axis=1)

    count=[[0]*total]*total
    count=np.array(count)
    #print(count)
    for i in range(0,len(test_data_y)):
      predictNew=maxIndex(predict[i])
      #print(str(test_data_y[i])+' -- '+ str(predictNew))
      #print()
      count[test_data_y[i]][predictNew]=count[test_data_y[i]][predictNew]+1

    with open(path +'/data/'+emotion+'_confution.csv',"w+") as my_csv:
      csvWriter = csv.writer(my_csv,delimiter=',')
      csvWriter.writerows(count)
    print(count)


##7 Emotion

In [ ]:
train_data_y = np.array(np.genfromtxt(path +'/data/train_data_y1.csv', delimiter=','))
#validation_data_y = np.array(np.genfromtxt(path +'/data/validation_data_y.csv', delimiter=','))
test_data_y = np.array(np.genfromtxt(path +'/data/test_data_y1.csv', delimiter=','))

train_data_y = to_categorical(train_data_y)
#validation_data_y = to_categorical(validation_data_y)
test_data_y = to_categorical(test_data_y)
emotion='7_emotion'
#acc,vacc=train(train_data_x, train_data_y, validation_data_x, validation_data_y,emotion,9)
acc=train(train_data_x, train_data_y,emotion,7)

emotion='7_emotion'
test_emotion(test_data_x, test_data_y,7,emotion)

In [ ]:
emotion='7_emotion'
test_emotion(train_data_x, train_data_y,7,emotion)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# use raw time-domain speech signal as input to cnn for SER

import numpy as np
import os
import librosa


import librosa
import pathlib
import numpy as np
from sklearn.model_selection import train_test_split
EmoDB_file_path = '/content/drive/Othercomputers/My Laptop 07-10-2021/MSc KUET/Thesis/Dataset/SUBESCO'
#EmoDB_file_path = '/content/drive/MyDrive/MSc KUET/Thesis/Dataset/SUBESCO' 
def get_log_mel_spectrogram(path, n_fft, hop_length, n_mels):
    """
    Extract log mel spectrogram
        1) The length of the raw audio used is 8s long,
        2) and then get the MelSpectrogram,
        2) finally perform logarithmic operation to MelSpectrogram.

    Return:
        log_mel_spectrogram:
    """
    y, sr = librosa.load(path, sr=16000, duration=3)

    file_length = np.size(y)
    if file_length != 128000:
        y = np.concatenate((y, np.zeros(128000-file_length)), axis=0)

        scal = np.arange(1, 129)  #513
    #mel_spectrogram, frequ = pywt.cwt(y, scal, "morl")
    #for i in range (0,128):
     #mel_spectrogram[i]=preprocessing.normalize([mel_spectrogram[i]])
      #max1=max(mel_spectrogram[i])
      #min1=min(mel_spectrogram[i])
      #mel_spectrogram[i]=(mel_spectrogram[i] - min1) / (max1 - min1)
      #print(mel_spectrogram[i])
    
    #print(mel_spectrogram)
    mel_spectrogram = librosa.feature.melspectrogram(y, sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    #n_fft=2048, hop_length=512, n_mels=128
    #print(shape(mel_spectrogram))
    #mel_spectrogram=np.abs(librosa.stft(y, n_fft=254))
    #print(shape(mel_spectrogram))
    #print("-------------------------------------------------------")
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram)
    log_mel_spectrogram = log_mel_spectrogram.reshape((-1,))

    return log_mel_spectrogram


def classify_files(path):
    """
    Classify emotion files and count them.
        Position 6 of emotion file name represent emotion which according to the label as follow:
        ( Emotion label letter used german word.)
        ----------------------------
           letter   |   emotion(En)
        ------------+---------------
              W     |   anger
              L     |   boredom
              E     |   disgust
              A     |   anxiety/fear
              F     |   happiness
              T     |   sadness
        ------------+---------------

    Dataset preprocessing.
        Dataset data are divided into

    Return:
        dataset_dict:
            a dict structure with 'total' used to count all file number, and a sub-dict named 'file_dict' which including three keys,
    """
    dataset_dict = {
        'total': 0,
        'file_dict': {
            'ANGRY': {'represent': 0, 'count': 0, 'all_data': []},
            'DISGUST': {'represent': 1, 'count': 0, 'all_data': []},
            'FEAR': {'represent': 2, 'count': 0, 'all_data': []},
            'HAPPY': {'represent': 3, 'count': 0, 'all_data': []},
            'NEUTRAL': {'represent': 4, 'count': 0, 'all_data': []},
            'SAD': {'represent': 5, 'count': 0, 'all_data': []},
            'SURPRISE': {'represent': 6, 'count': 0, 'all_data': []}
        }
    }

    wav_path = pathlib.Path(path)
    emotion_file_list = [str(file_name) for file_name in wav_path.glob('*.wav')]

    p = len(str(wav_path))
    #print(emotion_file_list)

    emotion_label_list = dataset_dict['file_dict'].keys()

    for emotion_label in emotion_label_list:
        #print(emotion_label)
        #print(emotion_file_list[0])
        #print(emotion_file_list[0].find(emotion_label))
        #actor = ['F_01','F_02','F_03','F_04','F_05','F_06','F_07','F_08','F_09','F_10','M_01','M_02','M_03','M_04','M_05','M_06','M_07','M_08','M_09','M_10']

        emotion_classify_file_list = [letter for letter in emotion_file_list if letter.find(emotion_label)!=-1 and (letter.find('F_02')!=-1 or letter.find('F_04')!=-1 or letter.find('F_06')!=-1 or letter.find('F_07')!=-1 or letter.find('M_03')!=-1 or letter.find('M_06')!=-1 or letter.find('M_09')!=-1 or letter.find('M_10')!=-1)]
        print(emotion_classify_file_list)
        files_count = len(emotion_classify_file_list)

        dataset_dict['file_dict'][emotion_label]['count'] = files_count
        dataset_dict['total'] = dataset_dict['total'] + files_count
        emotion_data = [get_log_mel_spectrogram(path, n_fft=2048, hop_length=512, n_mels=128)
                        for path in emotion_classify_file_list]
        dataset_dict['file_dict'][emotion_label]['all_data'] = emotion_data

    return dataset_dict


def load_data(path):
    """

    Returns:
        train_data_x, train_data_y:
            The emotion data and label of train data, which account for 80% in all.
        validation_data_x, validation_data_y:
            The emotion data and label of validation data, which account for 80% in train data.
        test_data_x, test_data_y:
            The emotion data and label of test data, which account for 20% in all.
    """
    train_data_x = []
    train_data_y = []
    validation_data_x = []
    validation_data_y = []
    test_data_x = []
    test_data_y = []

    dataset_dict = classify_files(path)

    '''Split data set'''
    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        x = dataset_dict['file_dict'][emotion_label]['all_data']
        count = dataset_dict['file_dict'][emotion_label]['count']
        y = np.full(count, dataset_dict['file_dict'][emotion_label]['represent'])

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8)

        train_data_x = np.append(train_data_x, x_train)
        train_data_y = np.append(train_data_y, y_train)

        validation_data_x = np.append(validation_data_x, x_val)
        validation_data_y = np.append(validation_data_y, y_val)

        test_data_x = np.append(test_data_x, x_test)
        test_data_y = np.append(test_data_y, y_test)

    '''Reshape all data'''
    train_data_x = np.array(train_data_x).reshape(-1, 128, 251, 1)
    train_data_y = np.array(train_data_y)
    validation_data_x = np.array(validation_data_x).reshape(-1, 128, 251, 1)
    validation_data_y = np.array(validation_data_y)
    test_data_x = np.array(test_data_x).reshape(-1, 128, 251, 1)
    test_data_y = np.array(test_data_y)

    return train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def model2d(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    model.add(layers.Reshape((-1, 128)))

    #LSTM
    model.add(layers.LSTM(256))

    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model



import tensorflow as tf
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


#physical_device = tf.config.experimental.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_device[0], True)




def train(train_data_x, train_data_y, validation_data_x, validation_data_y):
    model = model2d(input_shape=(128, 251, 1), num_classes=7)
    model.summary()
    es = EarlyStopping(monitor='val_loss',mode='min',verbose=0,patience=20)

    mc = ModelCheckpoint(EmoDB_file_path+'/model.h5',monitor='val_categorical_accuracy',mode='max',verbose=0,save_best_only=True)

    model.fit(train_data_x, train_data_y,validation_data=(validation_data_x, validation_data_y),epochs=200,batch_size=4,verbose=2,callbacks=[es, mc])

'''
def test(test_data_x, test_data_y ):
    new_model = load_model('model.h5')
    new_model.evaluate(test_data_x, test_data_y, batch_size=1)'''


if __name__ == '__main__':

    train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y = load_data(EmoDB_file_path)

    train_data_x = normalize(train_data_x)
    validation_data_x = normalize(validation_data_x)
    test_data_x = normalize(test_data_x)

    train_data_y = to_categorical(train_data_y)
    validation_data_y = to_categorical(validation_data_y)
    test_data_y = to_categorical(test_data_y)

    train(train_data_x, train_data_y, validation_data_x, validation_data_y)

    #test(test_data_x, test_data_y)


#EmoDB_file_path = '/content/drive/Othercomputers/My Laptop 07-10-2021/MSc KUET/Thesis/Dataset/SUBESCO'
#train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y = load_data(EmoDB_fi

SFFT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# use raw time-domain speech signal as input to cnn for SER

import numpy as np
import os
import librosa


import librosa
import pathlib
import numpy as np
from sklearn.model_selection import train_test_split
EmoDB_file_path = '/content/drive/Othercomputers/My Laptop 07-10-2021/MSc KUET/Thesis/Dataset/SUBESCO'
#EmoDB_file_path = '/content/drive/MyDrive/MSc KUET/Thesis/Dataset/SUBESCO' 
def get_log_mel_spectrogram(path, n_fft, hop_length, n_mels):
    """
    Extract log mel spectrogram
        1) The length of the raw audio used is 8s long,
        2) and then get the MelSpectrogram,
        2) finally perform logarithmic operation to MelSpectrogram.

    Return:
        log_mel_spectrogram:
    """
    y, sr = librosa.load(path, sr=16000, duration=3)

    file_length = np.size(y)
    if file_length != 128000:
        y = np.concatenate((y, np.zeros(128000-file_length)), axis=0)

        scal = np.arange(1, 129)  #513
    #mel_spectrogram, frequ = pywt.cwt(y, scal, "morl")
    #for i in range (0,128):
     #mel_spectrogram[i]=preprocessing.normalize([mel_spectrogram[i]])
      #max1=max(mel_spectrogram[i])
      #min1=min(mel_spectrogram[i])
      #mel_spectrogram[i]=(mel_spectrogram[i] - min1) / (max1 - min1)
      #print(mel_spectrogram[i])
    
    #print(mel_spectrogram)
    #mel_spectrogram = librosa.feature.melspectrogram(y, sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    #n_fft=2048, hop_length=512, n_mels=128
    #print(shape(mel_spectrogram))
    mel_spectrogram=np.abs(librosa.stft(y, n_fft=254))
    #print(shape(mel_spectrogram))
    #print("-------------------------------------------------------")
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram)
    log_mel_spectrogram = log_mel_spectrogram.reshape((-1,))

    return log_mel_spectrogram


def classify_files(path):
    """
    Classify emotion files and count them.
        Position 6 of emotion file name represent emotion which according to the label as follow:
        ( Emotion label letter used german word.)
        ----------------------------
           letter   |   emotion(En)
        ------------+---------------
              W     |   anger
              L     |   boredom
              E     |   disgust
              A     |   anxiety/fear
              F     |   happiness
              T     |   sadness
        ------------+---------------

    Dataset preprocessing.
        Dataset data are divided into

    Return:
        dataset_dict:
            a dict structure with 'total' used to count all file number, and a sub-dict named 'file_dict' which including three keys,
    """
    dataset_dict = {
        'total': 0,
        'file_dict': {
            'ANGRY': {'represent': 0, 'count': 0, 'all_data': []},
            'DISGUST': {'represent': 1, 'count': 0, 'all_data': []},
            'FEAR': {'represent': 2, 'count': 0, 'all_data': []},
            'HAPPY': {'represent': 3, 'count': 0, 'all_data': []},
            'NEUTRAL': {'represent': 4, 'count': 0, 'all_data': []},
            'SAD': {'represent': 5, 'count': 0, 'all_data': []},
            'SURPRISE': {'represent': 6, 'count': 0, 'all_data': []}
        }
    }

    wav_path = pathlib.Path(path)
    emotion_file_list = [str(file_name) for file_name in wav_path.glob('*.wav')]

    p = len(str(wav_path))
    #print(emotion_file_list)

    emotion_label_list = dataset_dict['file_dict'].keys()

    for emotion_label in emotion_label_list:
        #print(emotion_label)
        #print(emotion_file_list[0])
        #print(emotion_file_list[0].find(emotion_label))
        #actor = ['F_01','F_02','F_03','F_04','F_05','F_06','F_07','F_08','F_09','F_10','M_01','M_02','M_03','M_04','M_05','M_06','M_07','M_08','M_09','M_10']

        emotion_classify_file_list = [letter for letter in emotion_file_list if letter.find(emotion_label)!=-1 and (letter.find('F_02')!=-1 or letter.find('F_04')!=-1 or letter.find('F_06')!=-1 or letter.find('F_07')!=-1 or letter.find('M_03')!=-1 or letter.find('M_06')!=-1 or letter.find('M_09')!=-1 or letter.find('M_10')!=-1)]
        print(emotion_classify_file_list)
        files_count = len(emotion_classify_file_list)

        dataset_dict['file_dict'][emotion_label]['count'] = files_count
        dataset_dict['total'] = dataset_dict['total'] + files_count
        emotion_data = [get_log_mel_spectrogram(path, n_fft=2048, hop_length=512, n_mels=128)
                        for path in emotion_classify_file_list]
        dataset_dict['file_dict'][emotion_label]['all_data'] = emotion_data

    return dataset_dict


def load_data(path):
    """

    Returns:
        train_data_x, train_data_y:
            The emotion data and label of train data, which account for 80% in all.
        validation_data_x, validation_data_y:
            The emotion data and label of validation data, which account for 80% in train data.
        test_data_x, test_data_y:
            The emotion data and label of test data, which account for 20% in all.
    """
    train_data_x = []
    train_data_y = []
    validation_data_x = []
    validation_data_y = []
    test_data_x = []
    test_data_y = []

    dataset_dict = classify_files(path)

    '''Split data set'''
    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        x = dataset_dict['file_dict'][emotion_label]['all_data']
        count = dataset_dict['file_dict'][emotion_label]['count']
        y = np.full(count, dataset_dict['file_dict'][emotion_label]['represent'])

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9)

        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9)

        train_data_x = np.append(train_data_x, x_train)
        train_data_y = np.append(train_data_y, y_train)

        validation_data_x = np.append(validation_data_x, x_val)
        validation_data_y = np.append(validation_data_y, y_val)

        test_data_x = np.append(test_data_x, x_test)
        test_data_y = np.append(test_data_y, y_test)

    '''Reshape all data'''
    train_data_x = np.array(train_data_x).reshape(-1, 128, 2032, 1)
    train_data_y = np.array(train_data_y)
    validation_data_x = np.array(validation_data_x).reshape(-1, 128, 2032, 1)
    validation_data_y = np.array(validation_data_y)
    test_data_x = np.array(test_data_x).reshape(-1, 128, 2032, 1)
    test_data_y = np.array(test_data_y)

    return train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def model2d(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    model.add(layers.Reshape((-1, 128)))

    #LSTM
    model.add(layers.LSTM(256))

    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model



import tensorflow as tf
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


#physical_device = tf.config.experimental.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_device[0], True)




def train(train_data_x, train_data_y, validation_data_x, validation_data_y):
    model = model2d(input_shape=(128, 2032, 1), num_classes=7)
    model.summary()
    es = EarlyStopping(monitor='val_loss',mode='min',verbose=0,patience=20)

    mc = ModelCheckpoint(EmoDB_file_path+'/model.h5',monitor='val_categorical_accuracy',mode='max',verbose=0,save_best_only=True)

    model.fit(train_data_x, train_data_y,validation_data=(validation_data_x, validation_data_y),epochs=200,batch_size=4,verbose=2,callbacks=[es, mc])

'''
def test(test_data_x, test_data_y ):
    new_model = load_model('model.h5')
    new_model.evaluate(test_data_x, test_data_y, batch_size=1)'''


if __name__ == '__main__':

    train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y = load_data(EmoDB_file_path)

    train_data_x = normalize(train_data_x)
    validation_data_x = normalize(validation_data_x)
    test_data_x = normalize(test_data_x)

    train_data_y = to_categorical(train_data_y)
    validation_data_y = to_categorical(validation_data_y)
    test_data_y = to_categorical(test_data_y)

    train(train_data_x, train_data_y, validation_data_x, validation_data_y)

    #test(test_data_x, test_data_y)


#EmoDB_file_path = '/content/drive/Othercomputers/My Laptop 07-10-2021/MSc KUET/Thesis/Dataset/SUBESCO'
#train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y = load_data(EmoDB_file_path)


In [ ]:
def test(test_data_x, test_data_y ):
    new_model = load_model(EmoDB_file_path+'/model.h5')
    new_model.evaluate(test_data_x, test_data_y, batch_size=2)
test(test_data_x, test_data_y )

7 Emotion Spiker Dependent

In [ ]:
from numpy.core.fromnumeric import shape
import librosa
import pathlib
import numpy as np
import pywt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')
EmoDB_file_path = '/content/drive/Othercomputers/My Laptop 07-10-2021/MSc KUET/Thesis/Dataset/SUBESCO'
def get_log_mel_spectrogram(path, n_fft, hop_length, n_mels):
    """
    Extract log mel spectrogram
        1) The length of the raw audio used is 8s long,
        2) and then get the MelSpectrogram,
        2) finally perform logarithmic operation to MelSpectrogram.

    Return:
        log_mel_spectrogram:
    """
    y, sr = librosa.load(path, sr=16000, duration=3)

    file_length = np.size(y)
    if file_length != 128000:
        y = np.concatenate((y, np.zeros(128000-file_length)), axis=0)

    scal = np.arange(1, 129)  #513
    #mel_spectrogram, frequ = pywt.cwt(y, scal, "morl")
    #for i in range (0,128):
     #mel_spectrogram[i]=preprocessing.normalize([mel_spectrogram[i]])
      #max1=max(mel_spectrogram[i])
      #min1=min(mel_spectrogram[i])
      #mel_spectrogram[i]=(mel_spectrogram[i] - min1) / (max1 - min1)
      #print(mel_spectrogram[i])
    
    #print(mel_spectrogram)
    #mel_spectrogram = librosa.feature.melspectrogram(y, sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    #n_fft=2048, hop_length=512, n_mels=128
    #print(shape(mel_spectrogram))
    mel_spectrogram=np.abs(librosa.stft(y, n_fft=254))
    #print(shape(mel_spectrogram))
    #print("-------------------------------------------------------")
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram)
    log_mel_spectrogram = log_mel_spectrogram.reshape((-1,))

    return log_mel_spectrogram


def classify_files(path,actor):
    """
    Classify emotion files and count them.
        Position 6 of emotion file name represent emotion which according to the label as follow:
        ( Emotion label letter used german word.)
        ----------------------------
           letter   |   emotion(En)
        ------------+---------------
              W     |   anger
              L     |   boredom
              E     |   disgust
              A     |   anxiety/fear
              F     |   happiness
              T     |   sadness
        ------------+---------------

    Dataset preprocessing.
        Dataset data are divided into

    Return:
        dataset_dict:
            a dict structure with 'total' used to count all file number, and a sub-dict named 'file_dict' which including three keys,
    """
    dataset_dict = {
        'total': 0,
        'file_dict': {
            'ANGRY': {'represent': 0, 'count': 0, 'all_data': []},
            'DISGUST': {'represent': 1, 'count': 0, 'all_data': []},
            'FEAR': {'represent': 2, 'count': 0, 'all_data': []},
            'HAPPY': {'represent': 3, 'count': 0, 'all_data': []},
            'NEUTRAL': {'represent': 4, 'count': 0, 'all_data': []},
            'SAD': {'represent': 5, 'count': 0, 'all_data': []},
            'SURPRISE': {'represent': 6, 'count': 0, 'all_data': []}
        }
    }

    wav_path = pathlib.Path(path)
    emotion_file_list = [str(file_name) for file_name in wav_path.glob('*.wav')]

    p = len(str(wav_path))
    #print(emotion_file_list)

    emotion_label_list = dataset_dict['file_dict'].keys()

    for emotion_label in emotion_label_list:
        #print(emotion_label)
        #print(emotion_file_list[0])
        #print(emotion_file_list[0].find(emotion_label))
        emotion_classify_file_list = [letter for letter in emotion_file_list if letter.find(emotion_label)!=-1 and letter.find(actor)!=-1]
        print(emotion_classify_file_list)
        files_count = len(emotion_classify_file_list)

        dataset_dict['file_dict'][emotion_label]['count'] = files_count
        dataset_dict['total'] = dataset_dict['total'] + files_count
        emotion_data = [get_log_mel_spectrogram(path, n_fft=2048, hop_length=512, n_mels=128)
                        for path in emotion_classify_file_list]
        dataset_dict['file_dict'][emotion_label]['all_data'] = emotion_data

    return dataset_dict


def load_data(path,actor):
    """

    Returns:
        train_data_x, train_data_y:
            The emotion data and label of train data, which account for 80% in all.
        validation_data_x, validation_data_y:
            The emotion data and label of validation data, which account for 80% in train data.
        test_data_x, test_data_y:
            The emotion data and label of test data, which account for 20% in all.
    """
    train_data_x = []
    train_data_y = []
    validation_data_x = []
    validation_data_y = []
    test_data_x = []
    test_data_y = []

    dataset_dict = classify_files(path,actor)

    '''Split data set'''
    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        x = dataset_dict['file_dict'][emotion_label]['all_data']
        count = dataset_dict['file_dict'][emotion_label]['count']
        y = np.full(count, dataset_dict['file_dict'][emotion_label]['represent'])

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8)

        train_data_x = np.append(train_data_x, x_train)
        train_data_y = np.append(train_data_y, y_train)

        validation_data_x = np.append(validation_data_x, x_val)
        validation_data_y = np.append(validation_data_y, y_val)

        test_data_x = np.append(test_data_x, x_test)
        test_data_y = np.append(test_data_y, y_test)

    '''Reshape all data'''
    train_data_x = np.array(train_data_x).reshape(-1, 128, 2032, 1)
    train_data_y = np.array(train_data_y)
    validation_data_x = np.array(validation_data_x).reshape(-1, 128, 2032, 1)
    validation_data_y = np.array(validation_data_y)
    test_data_x = np.array(test_data_x).reshape(-1, 128, 2032, 1)
    test_data_y = np.array(test_data_y)

    return train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def model2d(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    model.add(layers.Reshape((-1, 128)))

    #LSTM
    model.add(layers.LSTM(256))

    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model


import tensorflow as tf
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


#physical_device = tf.config.experimental.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_device[0], True)




def train(train_data_x, train_data_y, validation_data_x, validation_data_y):
    model = model2d(input_shape=(128, 2032, 1), num_classes=7)
    model.summary()
    es = EarlyStopping(monitor='val_loss',mode='min',verbose=0,patience=20)

    mc = ModelCheckpoint(EmoDB_file_path+'/model.h5',monitor='val_categorical_accuracy',mode='max',verbose=0,save_best_only=True)

    history=model.fit(train_data_x, train_data_y,validation_data=(validation_data_x, validation_data_y),epochs=200,batch_size=4,verbose=2,callbacks=[es, mc])
    acc=history.history['val_categorical_accuracy'][len(history.history['val_categorical_accuracy'])-1]
    vacc=history.history['categorical_accuracy'][len(history.history['categorical_accuracy']) - 1]
    return acc,vacc

def test(test_data_x, test_data_y ):
    new_model = load_model(EmoDB_file_path+'/model.h5')
    history=new_model.evaluate(test_data_x, test_data_y, batch_size=1)
    return history[1]


if __name__ == '__main__':
    result=[]
    actor = ['F_01','F_02','F_03','F_04','F_05','F_06','F_07','F_08','F_09','F_10','M_01','M_02','M_03','M_04','M_05','M_06','M_07','M_08','M_09','M_10']

    
    for i in range (0,20):
        train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y = load_data(EmoDB_file_path,actor[i])
        train_data_x = normalize(train_data_x)
        validation_data_x = normalize(validation_data_x)
        test_data_x = normalize(test_data_x)

        train_data_y = to_categorical(train_data_y)
        validation_data_y = to_categorical(validation_data_y)
        test_data_y = to_categorical(test_data_y)

        acc,vacc=train(train_data_x, train_data_y, validation_data_x, validation_data_y)
        tacc=test(test_data_x, test_data_y)
        print([acc,vacc,tacc])
        result.append([acc,vacc,tacc])
    print(result)

#EmoDB_file_path = 'D:\Education\MSc KUET\Thesis\Dataset\SUBESCO'
#actor='F_01'
#train_data_x, train_data_y, validation_data_x, validation_data_y, test_data_x, test_data_y = load_data(EmoDB_file_path,actor)

In [ ]:
count=1
for i in result:
  print('Actor ' + str(count)+' \t'+str(i[1])+'\t'+str(i[0])+'\t'+str(i[2]))
  count=count+1